# Hybrid pipeline + Gaussian Multiplicative Chaos (GMC) integration

This notebook integrates GMC ideas into the hybrid quantum-classical pipeline. It is self-contained and runnable in `RUN_MODE='local'`.

## Install (run if needed)
```bash
!pip install --quiet qiskit qiskit-aer torch scipy numpy matplotlib nbformat
```

## Overview

- Build truncated Hamiltonian and add GMC potential.
- KAN surrogate sampling, VQE with GMC regularizer.
- Krylov, FFT, multifractal diagnostics.
- Simulated QPE refinement.


In [ ]:

# Imports & basic config
import os, json, math, time
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from scipy.linalg import expm, eig, fractional_matrix_power
from scipy.signal import find_peaks
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Operator
from qiskit.primitives import Estimator
from qiskit.extensions import UnitaryGate
from qiskit.algorithms.optimizers import COBYLA

# Config
RUN_MODE = 'local'
SAVE_DIR = 'results_gmc'
os.makedirs(SAVE_DIR, exist_ok=True)

N_TRUNC = 8
N_QUBITS = int(np.ceil(np.log2(N_TRUNC)))
DIM = 2 ** N_QUBITS

KAN_INITIAL = 20
KAN_ITERS = 3
KAN_CAND_PER_ITER = 6
KAN_EPOCHS = 80
KAN_M_UNITS = max(40, 6 * (2 * N_QUBITS))

VQE_REFINES = 3
VQE_REFINED_ITERS = 80

KRYLOV_DIM = 4

T_MAX = 6.0
N_TIME = 256
DELTA_T = T_MAX / N_TIME

QPE_COUNT_QUBITS = 5
QPE_SHOTS = 1024

SEED = 123
np.random.seed(SEED)
torch.manual_seed(SEED)

# Build small H = (x p + p x)/2 truncated
def ladder_ops(N):
    a = np.zeros((N,N), dtype=complex)
    for n in range(N-1):
        a[n,n+1] = np.sqrt(n+1)
    return a, a.conj().T

a, adag = ladder_ops(N_TRUNC)
x = (a + adag)/np.sqrt(2)
p = 1j*(adag - a)/np.sqrt(2)
H_small = 0.5*(x @ p + p @ x)
H_small = 0.5*(H_small + H_small.conj().T)
H_pad = np.zeros((DIM, DIM), dtype=complex)
H_pad[:N_TRUNC, :N_TRUNC] = H_small
H_op = Operator(H_pad)

# GMC sampling helpers
def sample_log_correlated_field(npts, eps=1e-4, var=1.0, rng=None):
    if rng is None: rng = np.random.default_rng()
    xs = np.linspace(0,1,npts,endpoint=False)
    C = np.zeros((npts,npts), dtype=float)
    for i in range(npts):
        for j in range(npts):
            r = abs(xs[i]-xs[j])
            C[i,j] = -np.log(max(r, eps))
    C = C - C.mean(axis=0) - C.mean(axis=1)[:,None] + C.mean()
    C += np.eye(npts) * 1e-6
    scale = var * npts / np.trace(C)
    C = C * scale
    X = rng.multivariate_normal(mean=np.zeros(npts), cov=C)
    return X

def multiplicative_chaos_weight(X, gamma=0.7):
    var = np.var(X)
    w = np.exp(gamma * X - 0.5 * gamma**2 * var)
    return w / np.mean(w)

gamma = 0.8
rng = np.random.default_rng(SEED)
X = sample_log_correlated_field(N_TRUNC, rng=rng)
w = multiplicative_chaos_weight(X, gamma=gamma)
V_diag = w - np.mean(w)
lambda_scale = 0.12
H_small_pert = H_small + lambda_scale * np.diag(V_diag)

# Qiskit simulator & estimator
sim = AerSimulator(method='statevector')
est = Estimator(sim)

# Ansatz and energy evaluator
n_params = 2 * N_QUBITS
def ansatz_circuit(theta):
    qc = QuantumCircuit(N_QUBITS)
    for i in range(N_QUBITS):
        qc.ry(float(theta[i]), i)
        qc.rz(float(theta[i + N_QUBITS]), i)
    for i in range(N_QUBITS - 1):
        qc.cx(i, i+1)
    return qc

def eval_energy_theta_local(theta, H_operator=Operator(np.zeros((DIM,DIM)))):
    qc = ansatz_circuit(theta)
    res = est.run([qc], [H_operator]).result()
    return float(res.values[0].real)

# KAN surrogate
class KANSurrogate(nn.Module):
    def __init__(self, in_dim, m_units=80, uni_hidden=12):
        super().__init__()
        self.inner = nn.Linear(in_dim, m_units)
        self.uni_w = nn.Parameter(torch.randn(m_units, uni_hidden) * 0.01)
        self.uni_b = nn.Parameter(torch.zeros(m_units, uni_hidden))
        self.uni_out = nn.Parameter(torch.randn(m_units) * 0.01)
        self.outer = nn.Linear(m_units, 1)
    def forward(self, x):
        z = self.inner(x)
        z_exp = z.unsqueeze(-1)
        hidden = torch.tanh(z_exp * self.uni_w.unsqueeze(0) + self.uni_b.unsqueeze(0))
        hid_mean = hidden.mean(dim=-1)
        uni = hid_mean * self.uni_out.unsqueeze(0)
        out = self.outer(uni)
        return out.squeeze(-1)

def train_kan_and_propose(H_operator, initial_samples=KAN_INITIAL):
    Theta = []
    Evals = []
    for i in range(initial_samples):
        th = np.random.uniform(-np.pi, np.pi, size=(n_params,))
        e = eval_energy_theta_local(th, H_operator)
        Theta.append(th); Evals.append(e)
    X = torch.tensor(np.array(Theta), dtype=torch.float32)
    y = torch.tensor(np.array(Evals), dtype=torch.float32)
    kan = KANSurrogate(n_params, m_units=KAN_M_UNITS).to('cpu')
    opt = torch.optim.Adam(kan.parameters(), lr=1e-3)
    for ep in range(80):
        opt.zero_grad(); p = kan(X); loss = ((p - y)**2).mean(); loss.backward(); opt.step()
    # propose candidates
    candidates = []
    for r in range(KAN_CAND_PER_ITER):
        th_var = torch.tensor(np.random.uniform(-np.pi, np.pi, size=(1,n_params)), dtype=torch.float32, requires_grad=True)
        optx = torch.optim.Adam([th_var], lr=0.2)
        for s in range(80):
            optx.zero_grad(); out = kan(th_var); out.backward(); optx.step()
        candidates.append(th_var.detach().cpu().numpy().reshape(-1))
    for c in candidates:
        e = eval_energy_theta_local(c, H_operator)
        Theta = np.vstack([Theta, c]); Evals = np.hstack([Evals, e])
    return np.array(Theta), np.array(Evals), kan

H_op_pad = Operator(np.pad(H_small_pert, ((0,DIM-N_TRUNC),(0,DIM-N_TRUNC))))
Theta, Evals, kan_model = train_kan_and_propose(H_op_pad)

# VQE with GMC regularizer helpers
def multifractal_structure_from_eigvals(eigvals, q_list=[1,2], window_sizes=[2,4,8]):
    spacings = np.diff(np.sort(eigvals))
    N = len(spacings)
    S = {}
    for q in q_list:
        S[q] = []
        for w in window_sizes:
            vals = []
            for i in range(0, max(1, N-w)):
                mass = np.sum(spacings[i:i+w])
                vals.append(mass**q)
            S[q].append(np.mean(vals))
    return S

vals_ref = np.real_if_close(eigvals(H_small_pert))
vals_ref.sort()
S_target = multifractal_structure_from_eigvals(vals_ref, q_list=[1,2], window_sizes=[2,4,8])

from qiskit.algorithms.optimizers import COBYLA
def vqe_with_gmc_regularizer(init_theta, H_operator, alpha=0.6):
    def obj(x):
        e = eval_energy_theta_local(x, H_operator)
        qc = ansatz_circuit(x)
        sv = sim.run(qc).result().get_statevector(qc)
        v = sv[:N_TRUNC]; v = v / np.linalg.norm(v)
        block = [v]
        for k in range(1, KRYLOV_DIM):
            vv = H_small_pert @ block[-1]
            for prev in block:
                vv = vv - np.vdot(prev, vv) * prev
            if np.linalg.norm(vv) < 1e-12: break
            block.append(vv / np.linalg.norm(vv))
        K = len(block)
        S = np.zeros((K,K), dtype=complex); Hproj = np.zeros((K,K), dtype=complex)
        for i in range(K):
            for j in range(K):
                S[i,j] = np.vdot(block[i], block[j])
                Hproj[i,j] = np.vdot(block[i], H_small_pert @ block[j])
        try:
            S_inv_sqrt = fractional_matrix_power(S, -0.5)
            H_eff = S_inv_sqrt @ Hproj @ S_inv_sqrt
            eigs = np.real_if_close(eig(H_eff)[0])
        except Exception:
            eigs = np.array([e])
        S_obs = multifractal_structure_from_eigvals(eigs, q_list=[1,2], window_sizes=[2,4,8])
        reg = 0.0
        for q in S_target:
            tgt = np.array(S_target[q]); obs = np.array(S_obs[q])
            reg += np.mean((np.log(1e-12 + obs) - np.log(1e-12 + tgt))**2)
        return float((1-alpha)*e + alpha * reg)
    cobyla = COBYLA(maxiter=80)
    xopt, val, _ = cobyla.optimize(num_vars=n_params, objective_function=obj, initial_point=init_theta)
    return xopt, val

seed_idxs = np.argsort(Evals)[:VQE_REFINES]
refined_thetas = []
refined_vals = []
for idx in seed_idxs:
    init = Theta[idx]
    xopt, v = vqe_with_gmc_regularizer(init, H_op_pad, alpha=0.6)
    refined_thetas.append(xopt); refined_vals.append(v)

# FFT spectral reconstruction from refined states
U_step = expm(-1j * np.pad(H_small_pert, ((0,DIM-N_TRUNC),(0,DIM-N_TRUNC))) * (T_MAX / N_TIME))
fft_results = []
for i, theta in enumerate(refined_thetas):
    qc = ansatz_circuit(theta)
    sv = sim.run(qc).result().get_statevector(qc)
    cur = sv.copy(); C_t = np.zeros(N_TIME, dtype=complex)
    for n in range(N_TIME):
        C_t[n] = np.vdot(sv, cur)
        cur = U_step @ cur
    window = np.hanning(N_TIME)
    spectrum = np.fft.fftshift(np.fft.fft(C_t * window))
    freqs = np.fft.fftshift(np.fft.fftfreq(N_TIME, d=(T_MAX/N_TIME)))
    power = np.abs(spectrum)
    peaks, _ = find_peaks(power, height=np.max(power)*0.06)
    fft_results.append({'freqs':freqs, 'power':power, 'peaks':peaks})

# Save a compact summary
summary = {
    'S_target': {k:list(map(float,v)) for k,v in S_target.items()},
    'refined_vals': [float(x) for x in refined_vals],
    'fft_peaks': [[float(x) for x in fft_results[i]['freqs'][fft_results[i]['peaks'][:8]]] for i in range(len(fft_results))]
}
with open(os.path.join(SAVE_DIR,'summary_gmc.json'),'w') as f:
    json.dump(summary,f,indent=2)
print('Done — summary saved to', os.path.join(SAVE_DIR,'summary_gmc.json'))
